In [11]:
import os
import asyncio
from msgraph.generated.models.o_data_errors.o_data_error import ODataError
from msgraph import GraphServiceClient
from msgraph.generated.users.users_request_builder import UsersRequestBuilder
from azure.identity.aio import ClientSecretCredential
import json
import urllib.request
import urllib.parse
import pandas as pd
import re


In [12]:
def get_secret(secret_name):
    import os
    return os.environ.get(secret_name)

appId = get_secret('GRAPH_CLIENT_ID')
appSecret = get_secret('GRAPH_CLIENT_SECRET') 
tenantId = get_secret('GRAPH_CLIENT_TENANT')
print("If your secrets were initialized properly you should see your tenant id.\nTenant Id: %s " % tenantId)

If your secrets were initialized properly you should see your tenant id.
Tenant Id: c5263ca1-a2e1-42fd-81bb-effcd1666efd 


In [13]:
client_credential = ClientSecretCredential( tenantId, appId, appSecret )
scopes = ['https://graph.microsoft.com/.default']
client = GraphServiceClient(credentials=client_credential, scopes=scopes) 


In [14]:
from msgraph.generated.models.user_collection_response import UserCollectionResponse

async def get_users():
    query_params = UsersRequestBuilder.UsersRequestBuilderGetQueryParameters(
        # Only request specific properties
        select = ['displayName', 'id', 'mail'],
        # request 50 users at a time
        #top = 50,
        # Sort by display name
        orderby= ['displayName']
    )

    request_config = UsersRequestBuilder.UsersRequestBuilderGetRequestConfiguration(
        query_parameters=query_params
    )

    userId,displayName,emailAddress = [],[],[]

    # get the first page of results
    user_response = await client.users.get(request_configuration=request_config)

    for user in user_response.value:
        userId.append(user.id)
        displayName.append(user.display_name)
        emailAddress.append(user.mail)
    
    # get subsequent pages of results
    while user_response.odata_next_link:
        request_info = client.users.to_get_request_information(request_configuration=request_config)
        request_info.url = user_response.odata_next_link
        # Hint: Passing an empty error mapper might not be the ideal solution
        user_response = await client.request_adapter.send_async(request_info, UserCollectionResponse, {})
        for user in user_response.value:
            userId.append(user.id)
            displayName.append(user.display_name)
            emailAddress.append(user.mail)
    
    users_dict = {'userId':userId,'displayName':displayName,'emailAddress':emailAddress}
    return users_dict

users = await get_users()
users_df = pd.DataFrame(data=users) 
users_df.info()

 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   userId        403 non-null    object
 1   displayName   403 non-null    object
 2   emailAddress  394 non-null    object
dtypes: object(3)
memory usage: 9.6+ KB
